## Template for Loading and Training Model

This is a template for developing TensorFlow models using several high-level APIs.
<ul>
    <li><a href='https://www.tensorflow.org/api_docs/python/tf/contrib/training/HParams'>HParams</a></li>
    <li><a href='https://www.tensorflow.org/api_docs/python/tf/estimator/RunConfig'>RunConfig</a></li>
    <li><a href='https://www.tensorflow.org/api_docs/python/tf/contrib/data/Dataset'>Dataset</a> with an <a href='https://www.tensorflow.org/api_docs/python/tf/contrib/data/Iterator'>Iterator</a></li>
    <li><a href='https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator'>Estimators</a> - <a href='https://www.tensorflow.org/extend/estimators'>Creating Estimators</a></li>
</ul>

In [ ]:
# Imports
import tensorflow as tf
import numpy as np
import time, datetime, shutil, os, glob
from model import model_fn, params, config

In [ ]:
# Reproducibility
tf.set_random_seed(0) # Graph level random seed
np.set_random_seed(0)

In [ ]:
# Gather data file paths
train_dirs = glob.glob('../train/*')
train_files = []
for d in train_dirs:
    train_files.extend(glob.glob(d + '/*.tfrecord'))

print(str(len(train_files)) + ' train_files.')

valid_dirs = glob.glob('../valid/*')
valid_files = []
for d in valid_dirs:
    valid_files.extend(glob.glob(d + '/*.tfrecord'))

print(str(len(valid_files)) + ' valid_files.')
data_files = {'train' : train_files, 'valid' : valid_files}

In [ ]:
# Hyperparameters
ts = time.time()
timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H-%M-%S')
model_dir = '/tmp/' + timestamp
print('Output directory at:\n' + model_dir)

params = get_hyper_params()

In [ ]:
# Create Dataset
def parser(record):
    # Define features
    features={
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'image_raw': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64),
    }

    # Parse out data
    parsed = tf.parse_single_example(record, features)
    # Can now get feature with parsed['feature']
    
    # Data augmentation, etc
    
    return {'feature' : value }, label

def load_dataset(mode):
    # Load the dataset
    files = data_files[mode]
    dataset = tf.contrib.data.TFRecordDataset(files)

    # Parse the TFRecord entries
    dataset = dataset.map(parser, num_parallel_calls=2)

    # Shuffle the data if training
    if mode == 'train':
        dataset = dataset.shuffle(buffer_size=1000)
    
    # Batch the data
    dataset = dataset.batch(batch_size)

    return dataset

def dataset_input_fn(mode):
    dataset = load_dataset(mode)
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

In [ ]:
# Take a look at some of the data to make sure we are 
#   saving/reloading correctly
% matplotlib inline

features, labels = dataset_input_fn('train')
print('Features keys: ' + str(features.keys()))
print('Labels shape: ' + str(labels.shape) + '\n')

sess = tf.Session()

feats, labs = sess.run([features, labels])
print('Features: ' + str(features.keys()) + '\n')
print('Labels shape: ' + str(labels.shape))

for k in feats.keys():
    print(k + ' shape: ' + str(feats[k].shape) + ', dtype: ' + str(feats[k].dtype))

image = (feats['start'][0]) / 256.
label = labels[0]
print('First label: ' + str(label))

plt.imshow(frame)

In [ ]:
# Define Model
tf.reset_default_graph()
estimator = tf.estimator.estimator(model, model_dir, config, params)

In [ ]:
# Training/Evaluation Loops
print('tensorboard --logdir=' + str(output_dir))
for e in range(hyper_params.train_epochs):
    print('Epoch: ' + str(e))
    estimator.train(input_fn=lambda: dataset_input_fn('train'))
    estimator.evaluate(input_fn=lambda: dataset_input_fn('valid'))

In [ ]:
# Note: This won't work in a Jupyter Notebook because __file__ is not
#       defined, but should work in a pure .py python file. '''

# Find the path of the current running file (train script)
curr_path = os.path.realpath(__file__)

# Define the path of your factored out model.py file
model_file = '/some/path/model.py'

# Now copy the training script and the model file to 
#   model_dir -- the same directory specified when creating the Estimator
# Note: copy over more files if there are other important dependencies.
shutil.copy(curr_path, model_dir)
shutil.copy(model_path, model_dir)